In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import time

print(tf.__version__)

2.3.1


## 卷積神經網路（CNN）

### 載入資料

In [2]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的圖片預設為uint8（0-255的數字）。以下程式碼將其正規化到0-1之間的浮點數，並在最後增加一維作為顏色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 從資料集中隨機取出batch_size個元素並返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

### 定義模型

In [3]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,             # 卷積層神經元（卷積核）數目
            kernel_size=[5, 5],     # 接受區的大小
            padding='same',         # padding策略（vaild 或 same）
            activation=tf.nn.relu   # 激活函数
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 32]
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

## 模型的訓練

In [5]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

In [6]:
model = CNN()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [7]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)

for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)

    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: ,loss %f" % (batch_index, loss.numpy()))

    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

177: ,loss 0.000350
batch 5178: ,loss 0.001316
batch 5179: ,loss 0.000199
batch 5180: ,loss 0.000825
batch 5181: ,loss 0.000044
batch 5182: ,loss 0.000152
batch 5183: ,loss 0.004099
batch 5184: ,loss 0.006514
batch 5185: ,loss 0.000116
batch 5186: ,loss 0.006989
batch 5187: ,loss 0.062304
batch 5188: ,loss 0.046250
batch 5189: ,loss 0.000386
batch 5190: ,loss 0.001184
batch 5191: ,loss 0.000100
batch 5192: ,loss 0.109253
batch 5193: ,loss 0.000766
batch 5194: ,loss 0.000214
batch 5195: ,loss 0.001392
batch 5196: ,loss 0.001654
batch 5197: ,loss 0.000935
batch 5198: ,loss 0.002216
batch 5199: ,loss 0.000749
batch 5200: ,loss 0.061523
batch 5201: ,loss 0.000178
batch 5202: ,loss 0.002035
batch 5203: ,loss 0.004697
batch 5204: ,loss 0.000062
batch 5205: ,loss 0.207696
batch 5206: ,loss 0.000632
batch 5207: ,loss 0.000064
batch 5208: ,loss 0.000342
batch 5209: ,loss 0.004181
batch 5210: ,loss 0.000020
batch 5211: ,loss 0.014844
batch 5212: ,loss 0.004775
batch 5213: ,loss 0.005044
batch 52

KeyboardInterrupt: 

## 模型的評估

In [9]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)

for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
    
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.992100
